# Riak

In [ ]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Lectura de información en Pandas

Partimos del dataset normalizado y lo desnormalizamos para guardarlo todo junto en RIAK

In [ ]:
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos",  engine='openpyxl')
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros",  engine='openpyxl')
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

In [ ]:
df.head(5)

## Riak ...

In [ ]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

Utilizamos un bucket para guardar los movimientos en formato JSON y otro para guardar el importe acumulado por miembro.

En el bucket de movimientos utilizamos una clave autogenerada (no vamos a poder localizar los movimientos por clave) y en el bucket con el importe acumulado la clave es el nombre de la persona en formato HASH, para no tener problemas ni con los espacios del nombre ni con los acentos

Se van a crear los siguientes buckets:

<style>
table {float:left}
</style>

|bucket|Clave|Contenido|
|: -|-|-|
|movimientos|Marca temporal|Todos los datos del dataset en formnato JSON y desnormalizados|
|acum_importes|Código HASH de la persona que realiza el movimiento|Mapa con el nombre de la persona y el importe acumulado|

El bucket de movientos tendrá los siguientes índices:

|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|
|idx_miembro_bin|Nombre de la persona|Localizar movimientos de una persona concreta|

In [ ]:
BUCKET_MOVIMIENTOS = 'movimientos'
BUCKET_ACUM_IMPORTES = 'acum_importes'

bucket_mov = cliente.bucket(BUCKET_MOVIMIENTOS)
bucket_acum_importes = cliente.bucket_type('maps').bucket(BUCKET_ACUM_IMPORTES)

Función para eliminar los datos de un BUCKET de Riak

In [ ]:
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        for key in keys:
            bucket.delete(key)

In [ ]:
# Borramos los datos ..
drop_keys(bucket_mov)
drop_keys(bucket_acum_importes)

## Inserción de información en Riak

Para generar los datos en formato JSON partimos del DataFrame de Pandas y lo exportamos a JSON (en formato String) para luego cargarlo en un diccionario de Python

Observa que cuando se genera el dato en formato JSON, los campos fechas se almacenan como un [TIMESTAMP de unix](http://www.unixtimestamp.com), por lo que habría que volver a convertirlo a fecha según el caso

In [ ]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [ ]:
print(len(json_list))

In [ ]:
pp(json_list[0])

Vamos a utilizar el nombre de la persona como clave, por lo que necesitamos convertirla previamente a un código que no lleve ni espacios ni acentos.

Este es precisamente el objetivo de la siguiente función ...

In [ ]:
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

Recorremos el dataset de movimientos, almacenando la información tanto en el bucket de movimientos como en el bucket que acumula los movimientos por cliente.

En este último utilizamos una funcionalidad que nos da Riak para guardar la información tipificada. Utilizamos un mapa que tiene dos elementos: Un registro donde guardamos el nombre del cliente y un contador donde guardamos el importe (sin decimales)

In [ ]:
# Bucle de carga de datos ...
for movimiento_json in json_list:
    key = uuid.uuid4().hex
    hash_nombre =  hash_string(movimiento_json["nombre"])
    
    movimiento = bucket_mov.new(key, movimiento_json)
    movimiento.add_index('idx_miembro_bin', hash_nombre)
    movimiento.store()
    
    map_nombre = bucket_acum_importes.new(hash_nombre)
    map_nombre.registers['nombre'].assign(movimiento_json["nombre"])
    map_nombre.counters['importe'].increment(int(round(movimiento_json["importe"] * 100)))
    map_nombre.store()
    

In [ ]:
for key in bucket_mov.get_keys()[0:3]:
    data = bucket_mov.get(key).data
    print('%s -> %s' % (key, data))

In [ ]:
for key in bucket_acum_importes.get_keys()[0:3]:
    data = bucket_acum_importes.get(key)
    print('%s -> %s' % (key, data))

# Lectura de información en Riak

En la siguiente función partimos del objeto JSON que nos devuelve RIAK y generamos un DataFrame Pandas.

En el caso de la fecha, lo convertimos desde un UNIX Timestamp al formato correcto para Pandas

In [ ]:
def json_to_pandas(rows):
    # Convertimos el objeto JSON en un objeto pandas 
    df = pd.read_json(json.dumps(rows))
    
    if 'fecha' in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df = df.assign(fecha = pd.to_datetime(df.fecha, unit = 'ms'))

    return df

### Lectura del bucket de movimientos

Cargamos todos los movimientos de la base de datos y lo guardamos en un objeto Pandas, ya que hay ciertas preguntas que no pueden resolverse directamente por esta base de datos

Recuerda que RIAK permite obtener información de un clave, pero no le es posible devolver la información ordenada ...

In [ ]:
rows = []
for keys in bucket_mov.stream_keys():
    for key in keys:
        # print('Key %s' % key )
        rows.append(bucket_mov.get(key).data)
        
# Convertimos el objeto json en un objeto pandas 
df_movimientos = json_to_pandas(rows)

In [ ]:
df_movimientos.head()

### Los 10 movimientos mas caros

Esta consulta no la podemos contestar directamente con la Base de Datos por lo que nos apoyamos en un proceso en el cliente

In [ ]:
df_movimientos \
    .sort_values('importe', ascending=False)\
    .filter(['nombre', 'fecha', 'actividad_completa', 'importe']) \
    .head(10)

### Lectura a través de un índice

In [ ]:
rows = []
keys = bucket_mov.stream_index("idx_miembro_bin", hash_string(u"Ildefonso José Sánchez Barcoj"))
for keys in keys.results:
    for movimiento_key in keys:
        rows.append(bucket_mov.get(movimiento_key).data)
    
df = json_to_pandas(rows)

### Los movimientos de una persona concreta

Para obtener los movimientos de una persona en concreto si podemos utilizar el índice que habíamos creado Ad-hoc, aunque la ordenación se realiza en Pandas

In [ ]:
df \
    .sort_values('importe', ascending=False) \
    .filter(['nombre', 'fecha', 'actividad_completa', 'importe']) \
    .head(10)

### Lectura de una información agregada

La información agregada está en un mapa, por lo que tenemos que procesarla y convertirla a formato JSON.

In [ ]:
rows = []
for keys in bucket_acum_importes.stream_keys():
    #print(keys)
    for key in keys:
        map_nombre = bucket_acum_importes.get(key)
        rows.append({'nombre' : map_nombre.registers['nombre'].value,
                     "importe" : float(map_nombre.counters['importe'].value) / 100})
        
# Convertimos el objeto json en un objeto pandas 
df = json_to_pandas(rows)

### Las 10 personas que mas han gastado

Fácil si disponemos de un agregado. Utilizamos Pandas para ordenar la información ..,

In [ ]:
df.sort_values('importe', ascending=False).head(10)